# Lab 1: Independent Component Analysis

### Machine Learning 2 (2017/2018)

* The lab exercises should be made in groups of two people.
* The deadline is Thursday, April 19, 23:59.
* Assignment should be submitted through BlackBoard! Make sure to include your and your teammates' names with the submission.
* Attach the .IPYNB (IPython Notebook) file containing your code and answers. Naming of the file should be "studentid1\_studentid2\_lab#", for example, the attached file should be "12345\_12346\_lab1.ipynb". Only use underscores ("\_") to connect ids, otherwise the files cannot be parsed.

Notes on implementation:

* You should write your code and answers in an IPython Notebook: http://ipython.org/notebook.html. If you have problems, please ask.
* Use __one cell__ for code and markdown answers only!
    * Put all code in the cell with the ```# YOUR CODE HERE``` comment and overwrite the ```raise NotImplementedError()``` line.
    * For theoretical questions, put your solution using LaTeX style formatting in the YOUR ANSWER HERE cell.
* Among the first lines of your notebook should be "%pylab inline". This imports all required modules, and your plots will appear inline.
* Large parts of you notebook will be graded automatically. Therefore it is important that your notebook can be run completely without errors and within a reasonable time limit. To test your notebook before submission, select Kernel -> Restart \& Run All.

### Literature
In this assignment, we will implement the Independent Component Analysis algorithm as described in chapter 34 of David MacKay's book "Information Theory, Inference, and Learning Algorithms", which is freely available here:
http://www.inference.phy.cam.ac.uk/mackay/itila/book.html

Read the ICA chapter carefuly before you continue!

### Notation

$\mathbf{X}$ is the $M \times T$ data matrix, containing $M$ measurements at $T$ time steps.

$\mathbf{S}$ is the $S \times T$ source matrix, containing $S$ source signal values at $T$ time steps. We will assume $S = M$.

$\mathbf{A}$ is the mixing matrix. We have $\mathbf{X} = \mathbf{A S}$.

$\mathbf{W}$ is the matrix we aim to learn. It is the inverse of $\mathbf{A}$, up to indeterminacies (scaling and permutation of sources).

$\phi$ is an elementwise non-linearity or activation function, typically applied to elements of $\mathbf{W X}$.

### Code
In the following assignments, you can make use of the signal generators listed below.



In [ ]:
%pylab inline
import sys
assert sys.version_info[:3] >= (3, 6, 0), "Make sure you have Python 3.6 installed"

# Signal generators
def sawtooth(x, period=0.2, amp=1.0, phase=0.):
    return (((x / period - phase - 0.5) % 1) - 0.5) * 2 * amp

def sine_wave(x, period=0.2, amp=1.0, phase=0.):
    return np.sin((x / period - phase) * 2 * np.pi) * amp

def square_wave(x, period=0.2, amp=1.0, phase=0.):
    return ((np.floor(2 * x / period - 2 * phase - 1) % 2 == 0).astype(float) - 0.5) * 2 * amp

def triangle_wave(x, period=0.2, amp=1.0, phase=0.):
    return (sawtooth(x, period, 1., phase) * square_wave(x, period, 1., phase) + 0.5) * 2 * amp

def random_nonsingular_matrix(d=2):
    """
    Generates a random nonsingular (invertible) matrix of shape d*d
    """
    epsilon = 0.1
    A = np.random.rand(d, d)
    while abs(np.linalg.det(A)) < epsilon:
        A = np.random.rand(d, d)
    return A

def plot_signals(X, title="Signals"):
    """
    Plot the signals contained in the rows of X.
    """
    figure()
    
    for i in range(X.shape[0]):
        ax = plt.subplot(X.shape[0], 1, i + 1)
        plot(X[i, :])
        ax.set_xticks([])
        ax.set_yticks([])
    plt.suptitle(title)

In [ ]:

# Signal generators
def sawtooth(x, period=0.2, amp=1.0, phase=0.):
    return (((x / period - phase - 0.5) % 1) - 0.5) * 2 * amp

def sine_wave(x, period=0.2, amp=1.0, phase=0.):
    return np.sin((x / period - phase) * 2 * np.pi) * amp

def square_wave(x, period=0.2, amp=1.0, phase=0.):
    return ((np.floor(2 * x / period - 2 * phase - 1) % 2 == 0).astype(float) - 0.5) * 2 * amp

def triangle_wave(x, period=0.2, amp=1.0, phase=0.):
    return (sawtooth(x, period, 1., phase) * square_wave(x, period, 1., phase) + 0.5) * 2 * amp

def random_nonsingular_matrix(d=2):
    """
    Generates a random nonsingular (invertible) matrix of shape d*d
    """
    epsilon = 0.1
    A = np.random.rand(d, d)
    while abs(np.linalg.det(A)) < epsilon:
        A = np.random.rand(d, d)
    return A

def plot_signals(X, title="Signals"):
    """
    Plot the signals contained in the rows of X.
    """
    figure()
    
    for i in range(X.shape[0]):
        ax = plt.subplot(X.shape[0], 1, i + 1)
        plot(X[i, :])
        ax.set_xticks([])
        ax.set_yticks([])
    plt.suptitle(title)

The following code generates some toy data to work with.

In [ ]:
# Generate data
num_sources = 5
signal_length = 500
t = linspace(0, 1, signal_length)
S = np.c_[sawtooth(t), sine_wave(t, 0.3), square_wave(t, 0.4), triangle_wave(t, 0.25), np.random.randn(t.size)].T
plot_signals(S)

### 1.1 Make mixtures (5 points)
Write a function `make_mixtures(S, A)' that takes a matrix of source signals $\mathbf{S}$ and a mixing matrix $\mathbf{A}$, and generates mixed signals $\mathbf{X}$.

In [ ]:
### 1.1 Make mixtures
def make_mixtures(S, A):
    '''Creates mixtures X (MxT).
    
    Args:
        S: SxT matrix containing the sources
        A: MxS matrix containing the mapping from the sources
           to the output
           
    Returns:
        MxT matrix, the mixed signals X.
    '''
    return A@S

In [ ]:
### Test your function
np.random.seed(42)
A = random_nonsingular_matrix(d=S.shape[0])
X = make_mixtures(S, A)
plot_signals(X, "Mixtures")

assert X.shape == (num_sources, signal_length), "The shape of your mixed signals is incorrect"


### 1.2 Histogram (5 points)
Write a function `plot_histograms(X)` that takes a data-matrix $\mathbf{X}$ and plots one histogram for each signal (row) in $\mathbf{X}$. You can use the `np.histogram()` (followed by `plot`) or `plt.hist()` function. 

Plot histograms of the sources and the measurements.

In [ ]:
### 1.2 Histogram
def plot_histograms(X, fig=plt.figure()):
    '''Plot the histograms of the measured X variables.
    
    Args:
        X: (MxT) matrix containing the signals
        fig, optional: (matplotlib.pyplot.figure) 
            Figure to plot in.
    
    '''
    rows, cols = shape(X);
#     wh = int(np.ceil(np.sqrt(rows)));
    for i, row in enumerate(X):
        ax = fig.add_subplot(1, rows, i+1);
        plt.hist(row);
        if i > 0:
            plt.setp(ax.get_yticklabels(), visible=False);
        ax.set_xlim([-2.5, 2.5]);
    plt.tight_layout()
plot_histograms(X)

Which of these distributions (sources or measurements) tend to look more like Gaussians? Can you think of an explanation for this phenomenon? Why is this important for ICA?

The Gaussian noise source is obviously expected to create a distribution which tends to look like a Gaussian. The sawtooth and the triangle are expected to look uniform. The block wave is degenerated. And the sine wave like a half-pipe. The measurements will look like a combination of these, depending on the values in A. For ICA it is important to be able to separate the sources, sinve the Gaussian is rotation invariant, ICA will not work if all sources are Gaussian.

### 1.3 Implicit priors (20 points)
As explained in MacKay's book, an activation function $\phi$ used in the ICA learning algorithm corresponds to a prior distribution over sources. Specifically, $\phi(a) = \frac{d}{da} \ln p(a)$. For each of the following activation functions, *derive* the source distribution they correspond to.
$$\phi_0(a) = -\tanh(a)$$
$$\phi_1(a) = -a + \tanh(a)$$
$$\phi_2(a) = -a^3$$ 
$$\phi_3(a) = -\frac{6a}{a^2 + 5}$$

Give your answer without the normalizing constant, so an answer of the form $p(a) \propto \verb+[answer]+$ is ok.

\begin{align}
    p(a) & \propto \frac{1}{cosh(a)} \\
    p(a) & \propto cosh(a) \exp\left[-\frac{a^2}{2}\right] \\
    p(a) & \propto \exp\left[-\frac{a^4}{4}\right] \\
    p(a) & \propto \frac{1}{(a^2 +5)^3} 
\end{align}

In [ ]:
def phi_0(a):
    return -np.tanh(a)

def p_0(a):
    return 1/np.cosh(a)

In [ ]:
def phi_1(a):
    return -a + np.tanh(a)

def p_1(a):
    return np.cosh(a) * np.exp(-0.5*(a**2))

In [ ]:
def phi_2(a):
    return -a**3

def p_2(a):
    return np.exp(-0.25*(a**4))

In [ ]:
def phi_3(a):
    return -6*a / (a**2 + 5)

def p_3(a):
    return (a**2 + 5)**(-3)

In [ ]:
activation_functions = [phi_0, phi_1, phi_2, phi_3]
priors = [p_0, p_1, p_2, p_3]

a = np.linspace(-5, 5, 1000)
for prior in priors:
    assert prior(a).shape == (1000, ), "Wrong output shape"


In [ ]:
a = np.linspace(-5, 5, 10)
for act_f in activation_functions:
    print(act_f(a))

Plot the activation functions and the corresponding prior distributions, from $a = -5$ to $5$ (hint: use the lists defined in the cell above). Compare the shape of the priors to the histogram you plotted in the last question.

In [ ]:
### 1.3 Implicit priors (continued)
a = np.linspace(-5, 5, 1000)

# YOUR CODE HERE
def plot_actv(a, priors, fig=plt.figure()):
    '''Plot the activation functions with the corresponding prioirs
    
    Args:
        a: (list) the activation functions
        priors: (list of func) List with priors.
        fig, optional: (matplotlib.pyplot.figure) 
            Figure to plot in.
    
    '''
    rows = len(priors)
#     wh = int(np.ceil(np.sqrt(rows)));
    for i, prior in enumerate(priors):
        ax = fig.add_subplot(1, rows, i+1);
        ax.plot(a, prior(a))
        if i > 0:
            plt.setp(ax.get_yticklabels(), visible=False);
        ax.set_xlim([-5, 5]);
    plt.tight_layout()
plot_actv(a, activation_functions)

### 1.4 Whitening (15 points)
Some ICA algorithms can only learn from whitened data. Write a method `whiten(X)` that takes a $M \times T$ data matrix $\mathbf{X}$ (where $M$ is the dimensionality and $T$ the number of examples) and returns a whitened matrix. If you forgot what whitening is or how to compute it, various good sources are available online, such as http://courses.media.mit.edu/2010fall/mas622j/whiten.pdf. Your function should also center the data before whitening.

In [ ]:
### 1.4 Whitening
import numpy.matlib
def whiten(X):
    # center matrix
#     Xw = X
#     for i in range(X.shape[0]):
#         Xw[i] -= mean(Xw[i])
    Xw = (X.T - (mean(X, 1)).T).T
    # use SVD to perform whitening
    U, s, Vt = np.linalg.svd(Xw, full_matrices=False)
    return np.dot(U, Vt)

In [ ]:
### Test your function
Xw = whiten(X)
assert Xw.shape == (num_sources, signal_length), "The shape of your mixed signals is incorrect"


### 1.5 Interpret results of whitening (10 points)
Make 3 figures, one for the sources, one for measurements and one for the whitened measurements. In each figure, make $5 \times 5$ subplots with scatter plots for each pair of signals. Each axis represents a signal and each time-instance is plotted as a dot in this space. You can use the `plt.scatter()` function. Describe what you see.

Now compute and visualize the covariance matrix of the sources, the measurements and the whitened measurements. You can visualize each covariance matrix using this code:
```python
# Dummy covariance matrix C;
C = np.eye(5)  
ax = imshow(C, cmap='gray', interpolation='nearest')
```

In [ ]:
### 1.5 Interpret results of whitening
import itertools
def plot_signalpairs (X):
    for i,j in itertools.product(range(0,5), repeat=2):
        plt.subplot(5,5, i*5+j+1)
        x = X[i]
        y = X[j]
        plt.scatter(x,y)
        
figsize = (10, 10)
plt.figure(1, figsize=figsize)
plot_signalpairs(S)

plt.figure(2, figsize=figsize)
plot_signalpairs(X)

plt.figure(3, figsize=figsize)
Y = whiten(X)
plot_signalpairs(Y)

Are the signals independent after whitening?

Yes, whitening first decorrelates the data using the eigenvalue decomposition (or SVD in our case). No correlation means independence.

### 1.6 Covariance (5 points)
Explain what a covariant algorithm is. 

A covariant algorithm is a algorithm that gives the same results independent of the units in which quantities are measured.

### 1.7 Independent Component Analysis (25 points)
Implement the covariant ICA algorithm as described in MacKay. Write a function `ICA(X, activation_function, learning_rate)`, that returns the demixing matrix $\mathbf{W}$. The input `activation_function` should accept a function such as `lambda a: -tanh(a)`. Update the gradient in batch mode, averaging the gradients over the whole dataset for each update. Make it efficient, so use matrix operations instead of loops where possible (loops are slow in interpreted languages such as python and matlab, whereas matrix operations are internally computed using fast C code). Experiment with the learning rate and the initialization of $\mathbf{W}$. Your algorithm should be able to converge (i.e. `np.linalg.norm(grad) < 1e-5`) in within 10000 steps.

In [ ]:
### 1.7 Independent Component Analysis
def ICA(X, activation_function, learning_rate=0.08, max_iters=10000, print_every=100000):
    '''Applies Independent Component
    '''
    # Initializes weights
    num_sources, num_samples = shape(X)
    W = random_nonsingular_matrix(num_sources)
    
    i, grad_norm = 1, np.inf
    while i < max_iters and (grad_norm > 1e-5):
        # ICA algorithm
        A = W @ X
        Z = activation_function(A)
        Xn = W.T @ A
        dW = W + (Z @ Xn.T) / num_samples
        
        # norm of grad
        grad_norm = linalg.norm(dW)
        
        # update weights
        W += learning_rate * dW
        
        if not i % print_every:
            print('Iteration %i, Error: %.5f' % (i, grad_norm))
        i += 1
    if i == max_iters:
        print('Not converged.')
    else:
        print('Converged in %i iterations, final weight gradient norm %.5f' % (i, grad_norm))
    return W

In [ ]:
# We will test your function so make sure it runs with only X and phi as input, and returns only W
# Also it should converge for all activation functions

W_estimates = [ICA(Xw, activation_function=phi) for phi in activation_functions]
assert all([W_est.shape == (num_sources, num_sources) for W_est in W_estimates])


### 1.8 Experiments  (5 points)
Run ICA on the provided signals using each activation function $\phi_0, \ldots, \phi_3$ (or reuse `W_estimates`). Use the found demixing matrix $\mathbf{W}$ to reconstruct the signals and plot the retreived signals for each choice of activation function.

In [ ]:
# 1.8 Experiments
# YOUR CODE HERE
for W in W_estimates:
    plot_signals(W@Xw, title="Reconstructed signals")

In [ ]:
import time

def does_whitening_make_a_difference():
    # Does it make a difference (in terms of speed of convergence) 
    # if you whiten your data before running ICA?
    
    t_start = time.time()
    W_estimates = [ICA(Xw, activation_function=phi) for phi in activation_functions]
    time_whitening = time.time() - t_start
    print('Time it takes to converges with whitening: %s' % time_whitening)
    
    t_start = time.time()
    W_estimates = [ICA(X, activation_function=phi) for phi in activation_functions]
    time_no_whitening = time.time() - t_start
    print('Time it takes to converge without whitening: %s' % time_no_whitening)
    
    # Return True or False
    # YOUR CODE HERE
    return time_whitening < time_no_whitening

In [ ]:
assert type(does_whitening_make_a_difference()) == bool

### 1.9 Audio demixing (10 points)
The 'cocktail party effect' refers to the ability humans have to attend to one speaker in a noisy room. We will now use ICA to solve a similar but somewhat idealized version of this problem. The code below loads 5 sound files and plots them.

Use a random non-singular mixing matrix to mix the 5 sound files. You can listen to the results in your browser using `play_signals`, or save them to disk if this does not work for you. Plot histograms of the mixed audio and use your ICA implementation to de-mix these and reproduce the original source signals. As in the previous exercise, try each of the activation functions.

Keep in mind that this problem is easier than the real cocktail party problem, because in real life there are often more sources than measurements (we have only two ears!), and the number of sources is unknown and variable. Also, mixing is not instantaneous in real life, because the sound from one source arrives at each ear at a different point in time. If you have time left, you can think of ways to deal with these issues.

In [ ]:
import scipy.io.wavfile
from IPython.display import Audio, display, Markdown

# Save mixtures to disk, so you can listen to them in your audio player
def save_wav(data, out_file, rate):
    scaled = np.int16(data / np.max(np.abs(data)) * 32767)
    scipy.io.wavfile.write(out_file, rate, scaled)

# Or play them in your browser
def play_signals(S, sample_rate, title="Signals"):
    display(Markdown(title))
    for signal in S:
        display(Audio(signal, rate=sample_rate))

In [ ]:
# Load audio sources
source_files = ['beet.wav', 'beet9.wav', 'beet92.wav', 'mike.wav', 'street.wav']
wav_data = []
sample_rate = None
for f in source_files:
    sr, data = scipy.io.wavfile.read(f, mmap=False)
    if sample_rate is None:
        sample_rate = sr
    else:
        assert(sample_rate == sr)
    wav_data.append(data[:190000])  # cut off the last part so that all signals have same length

# Create source and measurement data
S_audio = np.c_[wav_data]
plot_signals(S_audio)
play_signals(S_audio, sample_rate)

In [ ]:
### 1.9 Audio demixing
# YOUR CODE HERE
A_audio = random_nonsingular_matrix(d=S_audio.shape[0])
X_audio = make_mixtures(S_audio, A_audio)
plot_signals(X_audio)
play_signals(X_audio, sample_rate)

In [ ]:
def plot_2signals(X1, X2, title1="Reconstructe signals", title2='Signals', figsize=(10, 5)):
    """Plot two signals contained in the rows of X1 and X2
    """
    fig = figure(figsize=figsize)
    assert X1.shape == X2.shape
    num_s = X1.shape[0]
    colors = ['blue', 'red']
    index = 0
    for i in range(X1.shape[0]):
        index += 1
        ax = plt.subplot(num_s, 2, index)
        plot(X1[i, :], color=colors[0])
        ax.set_xticks([])
        ax.set_yticks([])
        if i == 0:
            ax.set_title(title1)
        
        index += 1
        ax = plt.subplot(num_s, 2, index)
        plot(X2[i, :], color=colors[1])
        ax.set_xticks([])
        ax.set_yticks([])
        if i == 0:
            ax.set_title(title2)

In [ ]:
Xw_audio = whiten(X_audio)
W_audio_est = [ICA(Xw_audio, activation_function=phi) for phi in activation_functions]
for W in W_audio_est:
    plot_2signals(W@Xw_audio, S_audio)
#     play_signals(W@Xw_audio, sample_rate)

In [ ]:
# The best denoiing matrices were create with the first and last activation function
W = W_audio_est[0]
plot_2signals(W@Xw_audio, S_audio)
play_signals(W@Xw_audio, sample_rate)

In [ ]:
W = W_audio_est[3]
plot_2signals(W@Xw_audio, S_audio)
play_signals(W@Xw_audio, sample_rate)

Report your results. Using which activation functions ICA recovers the sources?

With activation functions $\phi_0(a) = -\tanh(a)$ and $\phi_3(a) = -\frac{6a}{a^2 + 5}$ the best demixing matrices were found. It is expected that $\phi_2(a) = -a^3$ is not a good choice for demixing, because it implies a rotation invariant prior $p_2(a) \propto \exp[-\frac{a^4}{4}]$ (a Gauss with a faster decay), as explained in chapter 34 of David MacKay's book "Information Theory, Inference, and Learning Algorithms". The other actication function ($\phi_1(a) = -a + \tanh(a)$) has a linear term, which implies a gaussian prior. (The exponent in $p_1(a) \propto cosh(a) \exp\left[-\frac{a^2}{2}\right]$.) Probably due to this term, ICP with this activation function also does not generate a good demixing matrix.

### 1.10 Excess Kurtosis (15 points)
The (excess) kurtosis is a measure of 'peakedness' of a distribution. It is defined as
$$
\verb+Kurt+[X] = \frac{\mu_4}{\sigma^4} - 3 = \frac{\operatorname{E}[(X-{\mu})^4]}{(\operatorname{E}[(X-{\mu})^2])^2} - 3
$$
Here, $\mu_4$ is known as the fourth moment about the mean, and $\sigma$ is the standard deviation.
The '-3' term is introduced so that a Gaussian random variable has 0 excess kurtosis.
We will now try to understand the performance of the various activation functions by considering the kurtosis of the corresponding priors, and comparing those to the empirical kurtosis of our data.

#### 1.10.1 (10 points)
First, compute analytically the kurtosis of the four priors that you derived from the activation functions before. You may find it helpful to use an online service such as [Wolfram Alpha](https://www.wolframalpha.com/) or [Integral Calculator](https://www.integral-calculator.com/) to (help you) evaluate the required integrals. Give your answer as both an exact expression as well as a numerical approximation (for example $\frac{\pi}{2} \approx 1.571$).

YOUR ANSWER HERE

In [ ]:
### Include your answer here (you can use math.gamma if needed)
def get_kurtosis():
    # Return a list with 4 numbers / expressions
    # return [0, 0, 0, 0]
    
    # YOUR CODE HERE
    raise NotImplementedError()
    

In [ ]:
# Let's check
kurtosis = get_kurtosis()
print (kurtosis)
assert len(kurtosis) == 4


#### 1.10.2 (5 points)
Now use the `scipy.stats.kurtosis` function, with the `fisher` option set to `True`, to compute the empirical kurtosis of the dummy signals and the real audio signals.

Can you use this data to explain the performance of the various activation functions on the synthetic and real data?

In [ ]:
### 1.10.2 Excess Kurtosis
# YOUR CODE HERE
from scipy.stats import kurtosis
for s in S:
    print(kurtosis(s, fisher=True))
plot_signals(S)
for s in S_audio:
    print(kurtosis(s, fisher=True))
plot_signals(S_audio)

# YOUR ANSWER HERE